In [1]:
import cv2
import argparse
import os
import glob
import yolo_detector as yolo
from sklearn import metrics

In [2]:
path = "/home/prom/work/other/opencv_study/rice_dnn"
weights = f"{path}/backup/yolov4-rice-split-dnsty_best.weights"
config = f"{path}/cfg/yolov4-rice-split-dnsty.cfg"
classes = f"{path}/obj.names"
images_path = f"{path}/dense_and_sparse_label"
confidence = 0.4
nms = 0.4

In [3]:
detector = yolo.YoloDetector(weights, config, classes)

In [4]:
# f"{path}/test.txt"
test_images = open(file=f"{path}/test.txt").readlines()
test_files = [f"{path}/{line[:-1]}" for line in test_images]
# !cat "{path}/test.txt"

In [5]:
all_files = []
exts = ['*.jpg', '*.png']
all_files = [f for ext in exts for f in glob.glob(os.path.join(images_path, ext))]
# files = all_files
files = test_files


predict = []
actual = []

In [6]:


for file in files:
    image = cv2.imread(file)

    # labels = detector.detect_and_label(image, args.confidence, args.nms)
    classes, scores, boxes = detector.detect(image, confidence, nms)
    labels = detector.label(image)
    predict.append(classes[0])

    # print(file)
    path = os.path.splitext(file)
    if os.path.isfile(path[0]+'.txt'):
        txtfile = open(path[0]+'.txt','r')
        lines = txtfile.readlines()
        for line in lines:
            classid = int(line.split(' ')[0])
            if len(classes) <=0 or classid != classes[0]:
                is_match = False
            actual.append(classid)
            break



In [7]:
# actual
print(metrics.confusion_matrix(actual,predict))



[[16  2  0  0  0  0  0  0]
 [ 1 10  1  0  0  0  0  0]
 [ 0  1 12  0  0  0  0  0]
 [ 0  0  0 12  0  0  0  0]
 [ 0  0  0  0  1  0  0  0]
 [ 0  0  0  0  0  5  1  0]
 [ 0  0  0  0  0  0  5  0]
 [ 0  0  0  0  0  0  0  6]]


In [8]:
print(metrics.classification_report(actual, predict, target_names=detector.class_name))
# print(metrics.classification_report(actual, predict))

                 precision    recall  f1-score   support

 rice_100_dense       0.94      0.89      0.91        18
   rice_5_dense       0.77      0.83      0.80        12
  rice_10_dense       0.92      0.92      0.92        13
  rice_15_dense       1.00      1.00      1.00        12
rice_100_sparse       1.00      1.00      1.00         1
  rice_5_sparse       1.00      0.83      0.91         6
 rice_10_sparse       0.83      1.00      0.91         5
 rice_15_sparse       1.00      1.00      1.00         6

       accuracy                           0.92        73
      macro avg       0.93      0.93      0.93        73
   weighted avg       0.92      0.92      0.92        73

